In [17]:
#Install Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [23]:
#Upload Data
datafile = "/Users/justinpassek/Downloads/481802_SoFi_Creative_Fatigue_Testing_20191211_100848_2743124657.csv"
DF = pd.read_csv(datafile
               ,skiprows=10
                )
print(DF)

                              Campaign                  Creative        Date  \
0       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
1       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
2       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
3       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
4       A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2  2019-09-16   
...                                ...                       ...         ...   
109624                    SoFi H1 2017         Tracking Creative  2019-12-06   
109625                    SoFi H1 2017         Tracking Creative  2019-12-10   
109626             Turbo Affiliate Tag             2967 Tracking  2019-09-30   
109627             Turbo Affiliate Tag             2968 Tracking  2019-09-30   
109628                    Grand Total:                       ---         ---   

       Creative ID  Impressions   Click

In [24]:
#Preprocess Data
#df with Important Columns: 'Creative_id' , 'Date' , 'General_bucket' , 'CTR'
#A) Change Column Names. old_name:new_name
new_names = {'Campaign':'Campaign',
            'Creative ID':'Creative_id',
            'Date':'Date',
#            'old_name':'General_bucket',
            'Impressions':'Impressions',
            'Clicks':'Clicks'
            }
DF.rename(columns=new_names,inplace=True)

#B) Clean out 'Creative_id' without Adequate Metrics
#20 Impression Minimum 
DF = DF[DF["Impressions"]>1000]

#1 Click minimum
DF = DF[DF["Clicks"]>1]

#'Impressions' Must be Greater than the Clicks
DF = DF[DF["Impressions"]>DF["Clicks"]]

#Remove Unidentifiable Values/Rows
DF = DF[DF.Creative_id != '(not set)']
DF = DF[DF.Creative_id != '---']

#C) Add 'CTR' Column
DF['CTR']=DF.Clicks / DF.Impressions

DF['Date'] = pd.to_datetime(DF['Date'])

print(DF)

                                          Campaign                  Creative  \
0                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
1                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
2                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
3                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
4                   A_BR_GOOG_YouTube-Anthem_Y19Q3  BR_Anthem-15_TRACK_1x1_2   
...                                            ...                       ...   
109483  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109484  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109485  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109486  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   
109487  SoFi - Paid Social - Reddit - Q3FY19 - SLR                        H9   

             Date Creative_id  Impressi

In [25]:
#Text-to-Columns
Delimited_Campaign = DF['Campaign'].str.split('_', n=5, expand=True)
Delimited_Campaign.columns = ['Delimited_id{}'.format(x+1) for x in Delimited_Campaign.columns]

#Table with only the Relevant Columns
Delimited_Campaign = Delimited_Campaign[['Delimited_id1',
                                        'Delimited_id2',
                                        'Delimited_id4']]

#Rename Columns
Delimited_Campaign.rename(columns={"Delimited_id1":"Campaign_Type",
                                  "Delimited_id2":"Product_Line",
                                  "Delimited_id4":"Evergreen"},inplace=True)

#print(Delimited_Campaign)
#print(DF)

#Join Relevant Delimited Columns back to Table
DF = DF.join(Delimited_Campaign)

#print(DF)

#Convert String to Title-Style to Encompass all Instances of Evergreen
DF['Evergreen'] = DF['Evergreen'].str.title()
DF = DF[DF.Evergreen == 'Evergreen']

#Remove 'C' Campaign Types
DF = DF[DF.Campaign_Type != 'C']
print(DF)

                                 Campaign  \
7579    P_Brokerage_DV360_Evergreen_Y19Q1   
7580    P_Brokerage_DV360_Evergreen_Y19Q1   
7581    P_Brokerage_DV360_Evergreen_Y19Q1   
7582    P_Brokerage_DV360_Evergreen_Y19Q1   
7583    P_Brokerage_DV360_Evergreen_Y19Q1   
...                                   ...   
108358        R_SLR_DV360_Evergreen_Y19Q1   
108360        R_SLR_DV360_Evergreen_Y19Q1   
108361        R_SLR_DV360_Evergreen_Y19Q1   
108442        R_SLR_DV360_Evergreen_Y19Q1   
108443        R_SLR_DV360_Evergreen_Y19Q1   

                                                 Creative       Date  \
7579                    Brokerage_N/A_Panorma_5.14_C1TILE 2019-06-11   
7580                    Brokerage_N/A_Panorma_5.14_C1TILE 2019-06-12   
7581                    Brokerage_N/A_Panorma_5.14_C1TILE 2019-06-13   
7582                    Brokerage_N/A_Panorma_5.14_C1TILE 2019-06-14   
7583                    Brokerage_N/A_Panorma_5.14_C1TILE 2019-06-15   
...                        

In [29]:
df = DF
#Table_1 Granularity: 'Creative_id' level
imp_columns_1 = ['Campaign_Type',
                 "Product_Line",
               "Creative_id",
               "Date",
               "CTR",
               "Impressions",
               "Clicks"
                ]
Table_1 = df[imp_columns_1]

Table_1_R = Table_1[Table_1.Campaign_Type == 'R']

Table_1_P = Table_1[Table_1.Campaign_Type == 'P']

#print(Table_1)

#Table_2 with Important Columns: 'Date' , 'Impressions' , 'Clicks' , 'CTR',"Product_Line"
imp_columns_2 = ["Date",
                 "Impressions",
                 "Clicks",
                 "Campaign_Type"
                ]
Table_2 = df[imp_columns_2]

#Prospecting Only
Table_2 = Table_2[Table_2.Campaign_Type == 'P']

#Table_2 Granularity: Date level
Table_2 = Table_2.groupby(['Date']).sum()

#Reset (remove) Date from Index to Column
Table_2.reset_index(level=0, inplace=True)

#Add 'CTR' Column
Table_2['CTR'] = Table_2.Clicks / Table_2.Impressions

Prospecting = Table_2
print(Prospecting)

#Table_3
Table_3 = df[imp_columns_2]

#Remarketing Only
Table_3 = Table_3[Table_3.Campaign_Type == 'R']

#Table_3 Granularity: Date level
Table_3 = Table_3.groupby(['Date']).sum()

#Reset (remove) Date from Index to Column
Table_3.reset_index(level=0, inplace=True)

#Add 'CTR' Column
Table_3['CTR'] = Table_3.Clicks / Table_3.Impressions

Remarketing = Table_3

print(Remarketing)

          Date  Impressions  Clicks       CTR
0   2019-06-11       368752     323  0.000876
1   2019-06-12       632309     730  0.001154
2   2019-06-13       710678     946  0.001331
3   2019-06-14       757878    1089  0.001437
4   2019-06-15       772100    1214  0.001572
..         ...          ...     ...       ...
178 2019-12-06      1280989     431  0.000336
179 2019-12-07      1268753     509  0.000401
180 2019-12-08      1490622     515  0.000345
181 2019-12-09      1676826     520  0.000310
182 2019-12-10      1874478     548  0.000292

[183 rows x 4 columns]
          Date  Impressions  Clicks       CTR
0   2019-06-11       318138     256  0.000805
1   2019-06-12       348371     280  0.000804
2   2019-06-13       371135     372  0.001002
3   2019-06-14       390449     371  0.000950
4   2019-06-15       309119     456  0.001475
..         ...          ...     ...       ...
178 2019-12-06       405554     298  0.000735
179 2019-12-07       341033     341  0.001000
180 2019-1

In [30]:
#Table_1 Add Column: '7_day_MA'
Table_1_R['7_day_MA'] = Table_1.CTR.rolling(window=7).mean()
Table_1_P['7_day_MA'] = Table_1.CTR.rolling(window=7).mean()

#Table_2 Add Column: '90_day_MA'
Prospecting['90_day_MA'] = Prospecting.CTR.rolling(window=90).mean()
Remarketing['90_day_MA'] = Remarketing.CTR.rolling(window=90).mean()

#Join '90_day_MA' to Table_1 on 'Date'
Prospecting = Prospecting[['Date',
                  '90_day_MA'
                  ]]

Remarketing = Remarketing[['Date',
                  '90_day_MA'
                  ]]

Table_3_R = Table_1_R.join(Remarketing.set_index('Date'), on='Date')
Table_3_P = Table_1_P.join(Prospecting.set_index('Date'), on='Date')

Table_3 = pd.concat([Table_3_R, Table_3_P], ignore_index=True)

print(Table_3)

      Campaign_Type Product_Line Creative_id       Date       CTR  \
0                 R    Brokerage   116300917 2019-06-11  0.001693   
1                 R    Brokerage   116300917 2019-06-12  0.002271   
2                 R    Brokerage   116300917 2019-06-13  0.001898   
3                 R    Brokerage   116300917 2019-06-14  0.003100   
4                 R    Brokerage   116300917 2019-06-15  0.001233   
...             ...          ...         ...        ...       ...   
26600             P          SLR   114845847 2019-07-16  0.000404   
26601             P          SLR   114845847 2019-07-19  0.000790   
26602             P          SLR   114845847 2019-07-21  0.000657   
26603             P          SLR   114845847 2019-07-28  0.000906   
26604             P          SLR   114845847 2019-07-30  0.000978   

       Impressions  Clicks  7_day_MA  90_day_MA  
0             4135       7  0.000837        NaN  
1             4404      10  0.001100        NaN  
2             5796   

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
#Add Column to Int_Table_1 Calculating "Lift" by 'Creative_id' and 'Date'
Int_Table_1 = Table_3
Int_Table_1['MA_Diff_7-90'] = Int_Table_1['7_day_MA'] - Int_Table_1['90_day_MA']
print(Int_Table_1)

#Derive column "Max_Lift" in Int_Table_2
#Int_Table_2 = Int_Table_1
Int_Table_2 = Int_Table_1.groupby('Creative_id', as_index=False)['MA_Diff_7-90'].max()
Int_Table_2.rename(columns={"MA_Diff_7-90":"Max_Lift"},inplace=True)
Int_Table_2 = Int_Table_1.join(Int_Table_2.set_index('Creative_id'), on='Creative_id')
print(Int_Table_2)

#Derive column "Max_Date"
Int_Table_3 = Int_Table_2.loc[Int_Table_2['MA_Diff_7-90'] == Int_Table_2['Max_Lift']]
Int_Table_3.rename(columns={"Date":"Max_Date"},inplace=True)
print(Int_Table_3)

#Join 'Max_Date'
Int_Table_4 = Int_Table_1.join(Int_Table_3[['Creative_id','Max_Date']].set_index('Creative_id'), on='Creative_id')
print(Int_Table_4)

      Campaign_Type Product_Line Creative_id       Date       CTR  \
0                 R    Brokerage   116300917 2019-06-11  0.001693   
1                 R    Brokerage   116300917 2019-06-12  0.002271   
2                 R    Brokerage   116300917 2019-06-13  0.001898   
3                 R    Brokerage   116300917 2019-06-14  0.003100   
4                 R    Brokerage   116300917 2019-06-15  0.001233   
...             ...          ...         ...        ...       ...   
26600             P          SLR   114845847 2019-07-16  0.000404   
26601             P          SLR   114845847 2019-07-19  0.000790   
26602             P          SLR   114845847 2019-07-21  0.000657   
26603             P          SLR   114845847 2019-07-28  0.000906   
26604             P          SLR   114845847 2019-07-30  0.000978   

       Impressions  Clicks  7_day_MA  90_day_MA  MA_Diff_7-90  
0             4135       7  0.000837        NaN           NaN  
1             4404      10  0.001100       

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [33]:
#Filter for rows with 'Date' After 'Max_Date'
Int_Table_5 = Int_Table_4[Int_Table_4.Date > Int_Table_4.Max_Date]
#print(Int_Table_5)

#Filter for Rows with Negative Values
Int_Table_5 = Int_Table_5[Int_Table_5['MA_Diff_7-90'] <.0000000001]
#print(Int_Table_5)

#Find the Earliest 'Date'
Int_Table_5 = Int_Table_5.groupby('Creative_id').agg({'Date':np.min})

#Reset (remove) 'Creative_id' from Index to Column
Int_Table_5.reset_index(level=0, inplace=True)
Int_Table_5.rename(columns={"Date":"Intersection_Date"},inplace=True)
#print(Int_Table_5)

#Join Intersection Date to Creative level table
Int_Table_6 = Int_Table_5.join(Int_Table_3.set_index('Creative_id'), on='Creative_id')
print(Int_Table_6)

    Creative_id Intersection_Date Campaign_Type Product_Line   Max_Date  \
0     112371642        2019-09-14             R        Money 2019-09-13   
1     112373130        2019-09-09             R        Money 2019-09-08   
2     112373133        2019-09-13             R        Money 2019-09-12   
3     112373139        2019-09-13             R        Money 2019-09-12   
4     112392208        2019-09-16             R        Money 2019-09-15   
..          ...               ...           ...          ...        ...   
226   125233371        2019-12-02             P          PSL 2019-11-26   
227   125233728        2019-12-01             P          PSL 2019-11-28   
228   125234745        2019-12-02             P          PSL 2019-11-26   
229   125349835        2019-12-02             P          PSL 2019-11-26   
230   125349865        2019-12-09             P          PSL 2019-12-08   

          CTR  Impressions  Clicks  7_day_MA  90_day_MA  MA_Diff_7-90  \
0    0.001358         5155

In [42]:
#Derive column Decay
Int_Table_6['Decay'] = Int_Table_6['Intersection_Date'] - Int_Table_6['Max_Date']
Int_Table_6['Decay'] = Int_Table_6['Decay']/np.timedelta64(1,'D')
Final_Intersection_Table = Int_Table_6[['Creative_id',
                                        'Product_Line',
                                      'Campaign_Type',
                                      'Max_Lift',
                                       'Max_Date',
                                      'Intersection_Date',
                                      'Decay']]
#print(Final_Intersection_Table)

Final_Intersection_Table = Final_Intersection_Table[Final_Intersection_Table.Decay != '1.0']

print(Final_Intersection_Table)

    Creative_id Product_Line Campaign_Type  Max_Lift   Max_Date  \
0     112371642        Money             R -0.001284 2019-09-13   
1     112373130        Money             R -0.001159 2019-09-08   
2     112373133        Money             R -0.000702 2019-09-12   
3     112373139        Money             R -0.000814 2019-09-12   
4     112392208        Money             R -0.000813 2019-09-15   
..          ...          ...           ...       ...        ...   
226   125233371          PSL             P  0.000307 2019-11-26   
227   125233728          PSL             P  0.000354 2019-11-28   
228   125234745          PSL             P  0.000263 2019-11-26   
229   125349835          PSL             P  0.000227 2019-11-26   
230   125349865          PSL             P -0.000003 2019-12-08   

    Intersection_Date  Decay  
0          2019-09-14    1.0  
1          2019-09-09    1.0  
2          2019-09-13    1.0  
3          2019-09-13    1.0  
4          2019-09-16    1.0  
..       

/anaconda3/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [37]:
#Write file
Final_Intersection_Table.to_csv("/Users/justinpassek/Downloads/Final_Intersection_Table4.csv", index=False)